In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import multiprocessing as mp

from Bio.Align import MultipleSeqAlignment
from Bio import SeqIO, AlignIO, Seq, SeqRecord
import os
import re
import datetime
from pathlib import Path
import multiprocessing as mp

import numpy as np
import pandas as pd

# from Bio.Alphabet import Gapped
from Bio.Align import MultipleSeqAlignment
from Bio.Align import substitution_matrices as matlist
from Bio import Align
from Bio import SeqIO, AlignIO, Seq, Align, SeqRecord, pairwise2
from Bio import Phylo
import copy

/Users/varundeepakgudhe/opt/anaconda3/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
##DEL
fasta_handle="exp_seed/seed_BC.fa"

   
def performPairwiseAlns(ref_seq_record, target_fasta_handle):
    thres=100
    aln_score_list = []

    for target_seq in SeqIO.parse(target_fasta_handle, 'fasta'):

        if target_seq.seq==ref_seq_record.seq:
            continue  

        # Define pairwise alignment parameters
        aligner = Align.PairwiseAligner()
        aligner.open_gap_score = -10
        aligner.extend_gap_score = -0.5
        aligner.substitution_matrix = matlist.load('BLOSUM62')
        
        # Execute alignment
        print(ref_seq_record.seq,"\n", target_seq.seq,type(ref_seq_record.seq),type(target_seq.seq))
        break


    return aln_score_list, thres
for rec in SeqIO.parse(fasta_handle, 'fasta'):
    aln_score_list, thres=performPairwiseAlns(rec, fasta_handle)
    break
    print(aln_score_list, thres)
    
     

MTAFPFDSVMVANRGEIAARVLRTVKALGLRALLVAHRIDEGAPALMLADDVRWIDGPTPVAAFLDSQQIIAAAKDMGVGAIHPGYGFLSENADFARAVADAGMIFVGPTPAAIELMGDKVRARNFVAERGFPVAPSAIED--DDPATFVARARAVGAPLLIKPSAGGGGKGMRIVRDLATLDDEIVRARSEGQRYFGDGRLFVERYVERPRHIEVQVLGDAQGHVVHLFERECSLQRRFQKIVEEAPSPVLDDAERARICDAAAGIARAAGYVNAGTVEFIYGRGEFYFLEMNTRLQVEHPVTEAITGLDLVEQQLRIAAGLPLDFAQADVARTGHAIELRICAEDAGRDFAPTTGPVLKLTPPQG--ARFDVGVSEGGRVSAAFDPMIGKLIVHGTNRADALAKADAALGDLVLLGLKTNIAYLRRLIADPQVAAGDIHTGLIGEKVDIMPDPAPDDALLARLITLAAHHVPELVREAAEVPALHAAMGSWRN 
 ----MLDKIVIANRGEIALRILRACKELGIKTVAVHSSADRDLKHVLLADETVCIGPAPSVKSYLNIPAIISAAEITGAVAIHPGYGFLSENANFAEQVERSGFIFIGPKAETIRLMGDKVSAIAAMKKAGVPCVPGSDGPLGDDMDKNRAIAKRIGYPVIIKASGGGGGRGMRVVRGDAELAQSISMTRAEAKAAFSNDMVYMEKYLENPRHVEIQVLADGQGNAIYLAERDCSMQRRHQKVVEEAPAPGITPELRRYIGERCAKACVDIGYRGAGTFEFLFENGEFYFIEMNTRIQVEHPVTEMITGVDLIKEQLRIAAGQPLSIKQEEVHVRGHAVECRINAEDP-NTFLPSPGKITRFHAPGGFGVRWESHIYAGYTVPPYYDSMIGKLICYGENRDVAIARMKNALQELIIDGIKTNVDLQIRIMNDENFQHGGTNIHYLEKKLGLQEK----------------------------------------- <class

In [5]:
fasta_handle="exp_seed/seed_BC.fa"

   
def performPairwiseAlns(ref_seq_record, target_fasta_handle):
    thres=10000
    aln_score_list = []

    for target_seq in SeqIO.parse(target_fasta_handle, 'fasta'):
        if target_seq.seq==ref_seq_record.seq:
            continue
        # Define pairwise alignment parameters
        aligner = Align.PairwiseAligner()
        aligner.open_gap_score = -10
        aligner.extend_gap_score = -0.5
        aligner.substitution_matrix = matlist.load('BLOSUM62')
        
        # Execute alignment
        alignments = aligner.align(str(ref_seq_record.seq).replace("-",""),str(target_seq.seq).replace("-",""))
        best_aln = alignments[0]
        aln_score = best_aln.score
        thres=min(thres,aln_score)
        aln_score_list.append(aln_score)
        


    return aln_score_list, thres


min_thres=[]
mean_list=[]
for rec in SeqIO.parse(fasta_handle, 'fasta'):
    
    aln_score_list, thres=performPairwiseAlns(rec, fasta_handle)
    min_thres.append(thres)
    mean_list.append(sum(aln_score_list)/4)
    
print("min_thres::",min(min_thres),"\nmean_list::",mean_list)
    


min_thres:: 890.5 
mean_list:: [459.625, 579.125, 593.5]


### Therefore, the reference sequence is:

>311410_1:000749
----MFSKILIANRGEIALRILRACKELGIPTVAVHSTADADAMHVRLADESVCIGPPAA
RDSYLNIPQLLAACEITGADAVHPGYGFLSENARFAEILEAHNIEFIGPTAEHIRVMGDK
IQAKQTARDLGIPVVPGSDGEVTADD-DAHAIARQIGYPVLVKAAAGGGGRGMKVAKSEA
ELDTALSTARSEAKASFGDDALYMEKYLGKPRHIEIQVLGDGKGNAVHLGERDCSLQRRH
QKVLEEAPSPSLNAEQRHKIGQIVADAMRELKYRGVGTVEFLYENGEFYFIEMNTRLQVE
HPVTEMITGIDLVNEQIRIAAGGDLDMTQEDIKFDGHAIECRINAEDP-VTFAPSPGTIT
YYHPPGGLGVRVDSGVYQGYKIPPYYDSLIGKLIVHGRNRVECMMRLRRCLDEFVVDEIK
STIPLFRDLVDNQDIANGQYDIHWLEKYLADKSN--------------------------
---------------


### Threshold is 890.5

# Code to merge temperatures with the fasta file

In [26]:
liOrgListPath = 'Lietal_OrgTempList/temperature_data_full.tsv'

temped_Li_list = pd.read_csv(liOrgListPath, delimiter = '\t').drop_duplicates()
temped_Li_list['genus_name'] = temped_Li_list.organism.apply(lambda s: s.split('_')[0]).str.capitalize()
temped_Li_list['species_name'] = temped_Li_list.organism.apply(lambda s: s.split('_')[1])
temped_Li_list['genus_species'] = temped_Li_list.genus_name + ' ' + temped_Li_list.species_name

organism_temp_dict = temped_Li_list[['genus_species', 'temperature']].set_index('genus_species').squeeze().to_dict()

In [42]:
biotin_handle=Path("exp_seed/biotin_carboxylase_orthoDB.fasta")
new_biotin_handle=Path("exp_seed/biotin_carboxylase_orthoDB_temped.fasta")
with open(new_biotin_handle, 'a+') as nf:
    recs = []
    for record in SeqIO.parse(biotin_handle, 'fasta'):
        r = copy.copy(record)
        organism_raw = r.description.split(",")[4][17:-1]
        organism_split=organism_raw.split(" ")
        organism=organism_split[0]+" "+organism_split[1]
        try:
            r.id = ''
            r.name= ''
            r.description = 'Biotin carboxylase|{}|{}'.format(organism,organism_temp_dict[organism])
        except:
            continue
        recs.append(r)
    SeqIO.write(recs, nf, 'fasta')

## Code to eliminate records based on length of the sequence

#### The seeds sequence length is 495

In [51]:
biotin_handle_temped=Path("exp_seed/biotin_carboxylase_orthoDB_temped.fasta")
biotin_handle_eliminate=Path("exp_seed/biotin_carboxylase_orthoDB_temped_len_eliminate.fasta")
with open(biotin_handle_eliminate, 'a+') as nf:
    recs = []
    for record in SeqIO.parse(biotin_handle_temped, 'fasta'):
        r = copy.copy(record)
        if len(r.seq)>520:##There isnt much change even with +-20
            continue
        recs.append(r)
    SeqIO.write(recs, nf, 'fasta')

### Total sequences initially: 18696
### Total sequences after temped: 7034 (bcoz those organisms are not in our temp_org list)
### Total sequences after elimination based on length: 3155

# Code for pariwise alignment for all the sequences generated above with the  reference seq

In [61]:
def performPairwiseAlns(ref_seq_record, target_fasta_handle):
  
    aln_results = []
    count_in=0
    total=0
    for target_seq in SeqIO.parse(target_fasta_handle, 'fasta'):
        
        # Define pairwise alignment parameters
        aligner = Align.PairwiseAligner()
        aligner.open_gap_score = -10
        aligner.extend_gap_score = -0.5
        aligner.substitution_matrix = matlist.load('BLOSUM62')
        
        # Execute alignment
        alignments = aligner.align(str(ref_seq_record.seq).replace("-",""), str(target_seq.seq))
        best_aln = alignments[0]
        aln_score = best_aln.score
        
        total+=1
        if aln_score<890.5:
            continue
        count_in+=1

    print("Count of Total seq:",total,"\nCount of seq above threshold:",count_in)

target_fasta_handle=Path("exp_seed/biotin_carboxylase_orthoDB_temped_len_eliminate.fasta")

fasta_handle="exp_seed/seed_BC_reference.fa"
for rec in SeqIO.parse(fasta_handle, 'fasta'):
    performPairwiseAlns(rec, target_fasta_handle)

Count of Total seq: 3155 
Count of seq above threshold: 3115


In [62]:

def performPairwiseAlns(ref_seq_record, target_fasta_handle):
  
    aln_results = []

    for target_seq in SeqIO.parse(target_fasta_handle, 'fasta'):
        
        # Define pairwise alignment parameters
        aligner = Align.PairwiseAligner()
        aligner.open_gap_score = -10
        aligner.extend_gap_score = -0.5
        aligner.substitution_matrix = matlist.load('BLOSUM62')
        
        # Execute alignment
        alignments = aligner.align(str(ref_seq_record.seq).replace("-",""), str(target_seq.seq))
        best_aln = alignments[0]
        aln_score = best_aln.score
        
        if aln_score<890.5:
            continue

        aln_results.append(SeqRecord.SeqRecord(Seq.Seq(target_seq.seq), 
                                               id = '', 
                                               name = '', 
                                               description = target_seq.description
                                              )
                          )

    return aln_results
    
    
    
    
target_fasta_handle=Path("exp_seed/biotin_carboxylase_orthoDB_temped_len_eliminate.fasta")

fasta_handle="exp_seed/seed_BC_reference.fa"

for rec in SeqIO.parse(fasta_handle, 'fasta'):
    
    alignments =performPairwiseAlns(rec, target_fasta_handle)
    

out_aln_handle="exp_seed/biotin_carboxylase_orthoDB_thres_eliminate.fasta"

with open(out_aln_handle, 'a+') as alignment_file:
    SeqIO.write(alignments, alignment_file, 'fasta')

In [65]:
#Reducing sequences by 1 seq per spiece
import random 

def create_msas_by_organism(fasta_handle):
    retained_records = {}
    count=0
    for rec in SeqIO.parse(fasta_handle, 'fasta'):
        count+=1
        desc = rec.description.split('|')
        organism = desc[1]
        growth_temp = float(desc[2])
      
        if (organism,growth_temp) in retained_records.keys():
            retained_records[(organism, growth_temp)].append(rec)
        else:
            retained_records[(organism, growth_temp)] = [rec]
            
    for (organism, growth_temp) in retained_records.keys():
        retained_records[(organism, growth_temp)]=random.choices(retained_records[(organism, growth_temp)])
        
    print("original count:",count)
    return retained_records

newOut = Path('exp_seed/biotin_carboxylase_orthoDB_thres_eliminate.fasta')

result=create_msas_by_organism(newOut)

original count: 3115


In [66]:
#Saving the outputs
out_path = Path('exp_seed/biotin_carboxylase_orthoDB_one_seq_per_org.fasta')
with open(out_path, 'a+') as output:
    for i in result.values():
        SeqIO.write(i, output,'fasta')

In [67]:
#Checking the count of reduced sequences
reduced_count=0
for rec in SeqIO.parse(out_path, 'fasta'):
    reduced_count+=1
print("Reduced count of sequences above threshold:",reduced_count)    

Reduced count of sequences above threshold: 1806


In [68]:
#MAFFT
import subprocess

def add_sequences_to_alignment(new_sequences_file, seed_alignment_file, output_file):
  
    mafft_command = ["mafft", "--add", new_sequences_file, seed_alignment_file]
    
   
    with open(output_file, "w") as outfile:
        subprocess.run(mafft_command, stdout=outfile, check=True)


new_sequences = "exp_seed/biotin_carboxylase_orthoDB_one_seq_per_org.fasta"
seed_alignment = "exp_seed/seed_BC.fa"
output_alignment = "exp_seed/biotin_carboxylase_orthoDB_MAFFT.fasta"

add_sequences_to_alignment(new_sequences, seed_alignment, output_alignment)

nadd = 1806
nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8176 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 1801 / 1809
done.

Constructing a UPGMA tree (efffree=0) ... 
 1800 / 1809
done.

Progressive alignment 1/2... 
STEP  1701 / 1808  h
Reallocating..done. *alloclen = 2048
STEP  1801 / 1808  h
done.

Making a distance matrix from msa.. 
 1800 / 1809
done.

Constructing a UPGMA tree (efffree=1) ... 
 1800 / 1809
done.

Progressive alignment 2/2... 
STEP  1701 / 1808 
Reallocating..done. *alloclen = 2060
STEP  1801 / 1808  h
done.

disttbfast (aa) Version 7.520
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.

In [114]:
# #Modifing Description to contain only enzyme_name, growth_temperature,organism_name in the description.
# msa_handle=Path("seed_elimination/MAFFT/KSI_MAFFT.fa")
# out_path=Path("seed_elimination/MAFFT/Modified_Description/KSI_MAFFT.fa")
# count=0
# with open(out_path, 'a+') as output:
#     for rec in SeqIO.parse(msa_handle, 'fasta'):
#         if count<5:
#             count+=1
#             continue

        
#         desc=rec.description.split("|")
        
#         enzyme=desc[2]
#         temp=float(desc[4])
#         org=desc[3]
        
#         new_rec=SeqRecord.SeqRecord(Seq.Seq(rec.seq), id = '', name = '', 
#                                                description = '{}|{}|{}'.format(enzyme, org, temp))
        
#         SeqIO.write(new_rec, output,'fasta')
        

## Converting algined MAFFT sequences to csv

In [69]:
def msa_to_csv(msa_handle, out_root):
   
    consensus_msa = AlignIO.read(msa_handle, 'fasta')

    header_ids = ('Enzyme_Name', 'Organism', 'Temperature')

    msa_range = range(1, consensus_msa.get_alignment_length()+1)
    rec_df = pd.DataFrame([{
            **dict(zip(header_ids, rec.description.strip().split('|'))), 
            **dict(zip(msa_range, list(rec.seq)))} 
            for rec in consensus_msa]).melt(id_vars = header_ids, value_name = 'Consensus_Residue',
                                            value_vars = msa_range, var_name = 'Position')

 
    out_path = out_root.joinpath(msa_handle.stem + '_summary.csv') 
    rec_df.to_csv(out_path)
    
msa_handle=Path("exp_seed/biotin_carboxylase_orthoDB_MAFFT_ref_del.fasta")
out_root=Path("exp_seed")
msa_to_csv(msa_handle,out_root)

### UNABLE TO KEEP SEEDS IN THE CONSENSUS AND LOGISTIC REGRESSION, BECAUSE THEY DONT HAVE TEMP OR ATLEAST ORGANISM NAME TO FETCH THE TEMPERATURE FROM ORG TEMP DATA

## Code to convert MAFFT output sequences to contain only the amino acids at non gapped positions of the ACCC_ECOLI seq

In [10]:
aligned_ref_file = "exp_seed/seed_BC_ACCC_ECOLI.fa"
input_file = "exp_seed/biotin_carboxylase_orthoDB_MAFFT_ref_del.fasta"

def get_reference(ref_file):
    for rec in SeqIO.parse(ref_file,'fasta'):
        return rec
        
aligned_reference=get_reference(aligned_ref_file)


results=[]

for aligned_target in SeqIO.parse(input_file,"fasta"):

    aligned_reference_arr = np.array(list(aligned_reference), dtype = 'str')
    aligned_target_arr = np.array(list(aligned_target), dtype = 'str')
    trimmed_aligned_target = ''.join(aligned_target_arr[(aligned_reference_arr != '-')].tolist())

    # Append the record to our results list
    results.append(SeqRecord.SeqRecord(Seq.Seq(trimmed_aligned_target), 
                                           id = '', 
                                           name = '', 
                                           description = aligned_target.description
                                          ))
                   

In [12]:
output_file="exp_seed/biotin_carboxylase_orthoDB_MAFFT_trimmed_targets.fasta"
with open(output_file,'a+') as output:
    SeqIO.write(results, output_file,'fasta')


In [15]:
#Converting to csv file
msa_handle=Path("exp_seed/biotin_carboxylase_orthoDB_MAFFT_trimmed_targets.fasta")
out_root=Path("exp_seed")
msa_to_csv(msa_handle,out_root)